In [27]:
#dataset parameters
drawed = False

modality = "RGB"

#model name
model_name = "mobilenet"

#model parameters
width_mult = 2.0

dataset_drawed_path = "/home/diego/Scrivania/dataset/images_Drawed"

dataset_path = "/home/diego/Scrivania/dataset/GestureDataset/ego_gesture/images"

annotation_file_path = "./annotation_egogesture/egogestureall.json"

path_to_ckpt = "./ckpt/jester_"+model_name+"_"+str(width_mult)+"x_RGB_16_best.pth"


In [28]:
from utils.opts import parse_opts
from tqdm import tqdm

In [29]:
from dataset.spatial_transforms import *
from dataset.temporal_transforms import *
from dataset.target_transforms import ClassLabel, VideoID

spatial_transform = Compose([
            Scale(112),
            CenterCrop(112),
            ToTensor(1)
        ])

temporal_transform = TemporalCenterCrop(size=16, downsample = 1)

target_transform = ClassLabel()

In [30]:
from dataset.dataset import get_test_set, get_training_set

if drawed:
    path_to_dataset = dataset_drawed_path
else:
    path_to_dataset = dataset_path

assert modality in ['RGB', 'Depth'], "Modality not supported"

val_dataset = get_test_set(path_to_dataset,
                       annotation_file_path,
                       spatial_transform,
                       temporal_transform,
                       target_transform,
                       n_val_samples=1,
                       test_subset= "val",
                       modality=modality
                       )

train_dataset = get_training_set(video_path=path_to_dataset,
                       annotation_path= annotation_file_path,
                       spatial_transform=spatial_transform,
                       temporal_transform=temporal_transform,
                       target_transform=target_transform,
                       modality=modality
                       )

test_set = get_test_set(path_to_dataset,
                       annotation_file_path,
                       spatial_transform,
                       temporal_transform,
                       target_transform,
                       n_val_samples=1,
                       test_subset= "test",
                       modality=modality
                       )

[INFO]: EgoGesture Dataset - validation, is loading...
dataset loading [0/9929]
dataset loading [1000/9929]
dataset loading [2000/9929]
dataset loading [3000/9929]
dataset loading [4000/9929]
dataset loading [5000/9929]
dataset loading [6000/9929]
dataset loading [7000/9929]
dataset loading [8000/9929]
dataset loading [9000/9929]
[INFO]: EgoGesture Dataset - training, is loading...
dataset loading [0/30039]
dataset loading [1000/30039]
dataset loading [2000/30039]
dataset loading [3000/30039]
dataset loading [4000/30039]
dataset loading [5000/30039]
dataset loading [6000/30039]
dataset loading [7000/30039]
dataset loading [8000/30039]
dataset loading [9000/30039]
dataset loading [10000/30039]
dataset loading [11000/30039]
dataset loading [12000/30039]
dataset loading [13000/30039]
dataset loading [14000/30039]
dataset loading [15000/30039]
dataset loading [16000/30039]
dataset loading [17000/30039]
dataset loading [18000/30039]
dataset loading [19000/30039]
dataset loading [20000/30039

In [31]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

print("creating train dataloader...")
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=12)

print("creating val dataloader...")
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=12)

print("creating test dataloader...")
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=12)

creating train dataloader...
creating val dataloader...
creating test dataloader...


In [32]:
torch.cuda.is_available()

True

In [33]:
import torch
from utils.config import get_model

model = get_model(model_name, width_mult)


In [34]:
from torch import nn
import pytorch_lightning as pl
from models.custom_trainer import CustomTrainer

path = torch.load(path_to_ckpt, map_location=torch.device('cuda'))

model = nn.DataParallel(model, device_ids=[0],)

model.load_state_dict(path["state_dict"])


<All keys matched successfully>

In [35]:
from utils.config import get_classifier_layer

model.module.classifier = get_classifier_layer(model_name, width_mult)

model = model.cuda()

In [36]:
print(model)

DataParallel(
  (module): MobileNet(
    (features): Sequential(
      (0): Sequential(
        (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): Block(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=64, bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Block(
        (conv1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), groups=128, bias=False)
        (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(128, 2

In [37]:
torch.cuda.current_device()


0

In [38]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [39]:
from datetime import datetime 

from torch.utils.tensorboard import SummaryWriter
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10, verbose=False, mode="min")

EPOCHS = 100

model = model.cuda()

trainer = CustomTrainer(model = model, num_classes= 84)

trainer.fit(train_loader=train_loader, val_loader=val_loader, max_epochs=EPOCHS, callbacks=[early_stop_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | model    | DataParallel       | 13.0 M
1 | accuracy | MulticlassAccuracy | 0     
2 | ce_fn    | CrossEntropyLoss   | 0     
------------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
52.189    Total estimated model params size (MB)


Epoch 0:   9%|▊         | 20/235 [00:11<02:03,  1.74it/s, v_num=5]         

In [ ]:
trainer.test(test_loader = test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 81/81 [00:22<00:00,  3.57it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8267078399658203
        test_loss           0.7519878149032593
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.7519878149032593, 'test_acc': 0.8267078399658203}]